<a href="https://colab.research.google.com/github/maryantonopoulou/teliki_ergasia_EDDE2/blob/main/newsbomb_articles_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

import time
import random

from tqdm import tqdm

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_webpage_soup(url):
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  response = requests.get(url, headers=headers)
  if response.status_code != 200:
    return None
  else:
    return BeautifulSoup(response.text, 'html.parser')
def get_article_elements_from_soup(soup, article_url):
    # title
    try:
        title = soup.find("h1").text.strip()
    except:
        title = ""

    # datetime
    try:
        datetime_str = soup.find("time")["datetime"]
        date_str = datetime_str.split("T")[0]
        time_str = datetime_str.split("T")[1]
    except:
        date_str = ""
        time_str = ""
    # section
    try:
        section_div = soup.find("div", class_="main-category-block") or soup.find("div", class_="main-category")
        section = section_div.text.strip() if section_div else ""
    except:
        section = ""

    # author
    try:
      author_tag = soup.find("a", class_="author-name") or soup.find("span", class_="author-name")
      author = author_tag.text.strip() if author_tag else ""
    except:
        author = ""
    # excerpt
    try:
        excerpt_div = soup.find("div", class_="p.main-intro") or soup.find(itemprop="description") or soup.find("div", class_="nx-excerpt")
        excerpt = excerpt_div.text.strip() if excerpt_div else ""
    except:
        excerpt = ""

    # text body
    try:
      content_div = soup.find("div", class_="main-text story-fulltext")
      elements = content_div.find_all(["p", "h2", "h3", "ul", "ol", "li"]) if content_div else []
      text_body = " ".join(el.get_text(strip=True) for el in elements if el.get_text(strip=True))
    except:
        text_body = ""

    # feature image
    try:
        img_block = soup.find("div", class_="article-image") or soup.find("figure")
        feat_img_link = img_block.find("img")["src"] if img_block and img_block.find("img") else ""
    except:
        feat_img_link = ""

    # image caption
    try:
        caption_div = soup.find("div", class_="image-caption") or soup.find("figcaption")
        image_caption = caption_div.text.strip() if caption_div else ""
    except:
        image_caption = ""

    articleD = {
        "article_url": article_url,
        "title": title,
        "date": date_str,
        "time": time_str,
        "section": section,
        "author": author,
        "excerpt": excerpt,
        "text_body": text_body,
        "feat_img_link": feat_img_link,
        "feat_img_cation": image_caption
    }

    return articleD

In [ ]:
def scrape_newsbomb_articles_pages(teasers_df):
  all_articles_dataL = []
  base_url = "https://www.newsbomb.gr"
  for idx, row in tqdm(teasers_df.iterrows(), total=teasers_df.shape[0], desc="Processing rows"):
    article_url_relative = row['article_url']
    article_url = base_url + article_url_relative
    soup = get_webpage_soup(article_url)
    if soup:
     articleD = get_article_elements_from_soup(soup, article_url)
     all_articles_dataL.append(articleD)
    delay = random.uniform(1, 3)
    time.sleep(delay)
  df = pd.DataFrame(all_articles_dataL)
  print(f"\nScraping completed. Fetshed {len(df)} articles.")
  return df

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ergatika_atiximatadf=pd.read_csv('/content/drive/MyDrive/CSVFILES/newsbomb_teasers.csv')

In [ ]:
articles_df = scrape_newsbomb_articles_pages(ergatika_atiximatadf)
articles_df

Processing rows: 100%|██████████| 395/395 [21:42<00:00,  3.30s/it]


Scraping completed. Fetshed 395 articles.


,article_url,title,date,time,section,author,excerpt,text_body,feat_img_link,feat_img_cation
0,https://www.newsbomb.gr/ellada/story/1654458/e...,Eργατικό ατύχημα στη Σίνδο: Στη ΜΕΘ διασωληνωμ...,2025-06-04,13:14:44+03:00,ΕΛΛΑΔΑ,Newsbomb,Ο 34χρονος διακομίσθηκε σε σοβαρή κατάσταση κα...,Ενας34χρονος άνδρας τραυματίστηκε σοβαρά σεεργ...,https://img.bbmd.gr/img/1260/max/82/2024/07/28...,Φωτογραφία αρχείου
1,https://www.newsbomb.gr/ellada/story/1652901/v...,Βελεστίνο: Εργατικό ατύχημα σε εκκοκκιστήριο –...,2025-05-28,21:34:12+03:00,ΕΛΛΑΔΑ,Newsbomb,Ο τραυματίας μεταφέρθηκε αρχικά στο Κέντρο Υγε...,Σοβαράτραυματίαςείναι ένας 55χρονος άνδρας εργ...,https://img.bbmd.gr/img/1260/max/82/2024/06/26...,EUROKINISSI
2,https://www.newsbomb.gr/ellada/story/1652170/s...,Σίνδος: 45χρονος έπεσε από ύψος 3 μέτρων ενώ έ...,2025-05-26,11:41:36+03:00,ΕΛΛΑΔΑ,Newsbomb,Ο άνδρας μεταφέρθηκε με ασθενοφόρο του ΕΚΑΒ στ...,Εργατικό ατύχημα με έναν 45χρονο να τραυματίζε...,https://img.bbmd.gr/img/1260/max/82/2025/05/12...,
3,https://www.newsbomb.gr/ellada/story/1651530/t...,Θεσσαλονίκη: Εργατικό ατύχημα - 58χρονος έπεσε...,2025-05-23,12:18:01+03:00,ΕΛΛΑΔΑ,Newsbomb,Ο εργαζόμενος μεταφέρθηκε προληπτικά στο νοσοκ...,Εργατικό ατύχημασημειώθηκε το πρωί της Παρασκε...,https://img.bbmd.gr/img/1260/max/82/2025/05/23...,Voria.gr
4,https://www.newsbomb.gr/ellada/story/1650964/i...,Ηράκλειο: 30χρονος έπεσε από ύψος - Μεταφέρθηκ...,2025-05-21,10:30:23+03:00,ΕΛΛΑΔΑ,Newsbomb,Ο άνδρας γλίστρησε και έχασε την ισορροπία με ...,Εργατικό ατύχημασημειώθηκε το πρωί της Τετάρτη...,https://img.bbmd.gr/img/1260/max/82/2025/05/12...,
...,...,...,...,...,...,...,...,...,...,...
390,https://www.newsbomb.gr/ellada/story/32814/sob...,Σοβαρό εργατικό ατύχημα,2011-03-04,19:22:37+02:00,ΕΛΛΑΔΑ,Newsbomb,Σε σοβαρή κατάσταση νοσηλεύεται στην Μονάδα Εν...,Σε σοβαρή κατάσταση νοσηλεύεται στην Μονάδα Εν...,,
391,https://www.newsbomb.gr/ellada/story/24692/apo...,Αποζημίωση για εργατικό ατύχημα,2011-01-31,21:30:14+02:00,ΕΛΛΑΔΑ,Newsbomb,Αποζημίωση ύψους 50.000 ευρώ υποχρεώνεται να κ...,Αποζημίωση ύψους 50.000 ευρώ υποχρεώνεται να κ...,,
392,https://www.newsbomb.gr/ellada/story/24669/erg...,Εργατικό ατύχημα στον Σκαραμαγκά,2011-01-31,20:18:27+02:00,ΕΛΛΑΔΑ,Newsbomb,"Ένας 46χρονος εργάτης, τραυματίστηκε σοβαρά στ...","Ένας 46χρονος εργάτης, τραυματίστηκε σοβαρά στ...",,
393,https://www.newsbomb.gr/ellada/story/24655/sob...,Σοβαρό εργατικό ατύχημα σε φορτηγό πλοίο,2011-01-31,19:42:59+02:00,ΕΛΛΑΔΑ,Newsbomb,"Σήμερα στις 09.55, η Λιμενική Αρχή Χίου ενημερ...","Σήμερα στις 09.55, η Λιμενική Αρχή Χίου ενημερ...",,


In [ ]:
articles_df.to_csv('/content/drive/MyDrive/CSVFILES/newsbomb_articles.csv', index=False)